## How to access a Julia variable in PAN

In [ ]:
using PAN
using Random
using Plots
using Printf: @printf

The purpose of this notebook is to illustrate how a Julia variable can be used in a PAN netlist.

As an example, we will use the Van der Pol oscillator: for a detailed description of the model, see the [Van der Pol notebook](Van_der_Pol.ipynb).

The following netlist defines the Van der Pol circuit in PAN. Notice the presence of the <tt>port</tt> device, whose parameter <tt>noisesamples</tt> specifies the name of the variable containing the noisy samples. The content of this variable will be explained in the following. 

In [ ]:
netlist = """
ground electrical gnd

parameters G=1/10k

; Circuit

p1    x   gnd  x    gnd  poly n0=0 n1=-1 n2=0 n3=1 d0=1
c1    x   gnd  capacitor c=1 icon=200m
rx    x   gnd  RES  r=1/G
l1    x   gnd  inductor  l=1

ns    x   gns  port noisesamples="noisy_jl"

model RES resistor kf=0 af=1
""";

#### Write the above netlist to file

In [ ]:
netlist_file = "vanderpol_with_noise.pan"
open(netlist_file, "w") do io
    write(io, netlist)
end;

### Load the netlist with the description of the circuit
No analysis is performed at this point, since none is defined in the netlist.

In [ ]:
ok, libs = load_netlist(netlist_file)
if ! ok
    @printf("Problems loading netlist '%s'.\n", netlist_file)
end

### Build the variable to be used in PAN
The noise samples need to be stored in a matrix with two rows, representing time instants and noise values, respectively. The number of samples needs to be sufficient to cover the whole duration of the simulation: PAN will use the same noise value for those time instants that fall in between the ones provided by the user.

In [ ]:
dt = 0.1
tstop = 100
n_samples = Integer(tstop / dt) + 2
noisy_jl = zeros(2, n_samples)
noisy_jl[1,:] = 0 : dt : (tstop + dt)
noisy_jl[2,:] = 1000 * randn(n_samples);

### Run a transient analysis

Notice that the <tt>noiseinj=2</tt> option to the tran analysis specifies that noise is injected only in the (nonlinear) circuit and therefore affects exclusively the large signal solution. <br/>
Alternatively, one could use <tt>noiseinj=1</tt> to indicate that noise should be injected in the variational part of the circuit.

In [ ]:
tran_name = "Tr"
mem_vars = ["time", "x", "l1:i"]
data = tran(tran_name, tstop, mem_vars, libs, uic=1, noisefmax=10, noiseinj=2);

### Plot the results

In [ ]:
plot(data[:,1], data[:,2], color="black", lw=1, label="v_C [V]")
plot!(data[:,1], data[:,3], color="red", lw=1, label="i_L [A]")
xlabel!("Time [s]")